In [ ]:
import numpy as np
import itertools
import tensor_output_reader as tor
import pandas as pd
np.set_printoptions(
    linewidth=np.inf
)
np.set_printoptions(suppress=True,floatmode='maxprec')
# Source - https://stackoverflow.com/a
# Posted by simon, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-04, License - CC BY-SA 4.0

from IPython.display import display, HTML
display(HTML("<style>div.jp-OutputArea-output pre {white-space: pre;}</style>"))


In [ ]:
defout_ne = 'comp_between_default_and_standard_ne/default_check_for_fortran_printing.166751-20251205.out'
stdout_ne = 'comp_between_default_and_standard_ne/standard_check_for_cpp_printing.166752-20251205.out'
# Cu
defout_cu = 'comp_between_default_and_standard_cu/default_check_for_fortran_printing.165081-20251203.out'
stdout_cu = 'comp_between_default_and_standard_cu/standard_check_for_cpp_printing.165082-20251203.out'

defout_nh3 = 'comp_between_default_and_standard_nh3/default_check_for_fortran_printing.165078-20251203.out'
stdout_nh3 = 'comp_between_default_and_standard_nh3/standard_check_for_cpp_printing.165080-20251203.out'

## Tensor comparing functions

In [ ]:
def compare(tens1, tens2, tol = 1e-5):
    ind2s = []
    for ind1, val1, in np.ndenumerate(tens1):
        #space = 0
        if val1 == 0:
            continue
        ind1s = []
        for ind2, val2 in np.ndenumerate(tens2):
            if abs(val2 - val1) <= tol:
                #if abs(val2) < 1e-12: continue
                #if ind2 in ind2s: continue
                if ind1 in ind1s:
                    print(" ", ind2,)
                else:
                    print(ind1, ind2, val1, val2)
                ind1s.append(ind1)
                #space += 1
                #break
 
def compare_side_by_side(tens1, tens2, skip_match=False, skip_zeros=False, tol=1e-6):
    '''compare tens1 and tens2 index by index'''
    lines = ''
    zero_count = 0
    match_count = 0
    diff_count = 0
    non_zero_diff_count = 0
    zero_diff_count = 0
    
    for ind, val1 in np.ndenumerate(tens1):
        val2 = tens2[ind]

        equality = ''
        if val1 == 0:
            zero_count += 1
                
        if abs(val1 - val2) < tol:
            match_count += 1
            if skip_match: continue
            equality = '= ='
        else:
            diff_count += 1
            if val1 == 0:
                zero_diff_count += 1
            else:
                non_zero_diff_count += 1
            
            equality = '=/='

        if val1 == 0 and val2 == 0: 
            if skip_zeros:
                continue
                
        val1_str = f'{f"{0:^17d}" if val1 == 0 else f"{val1:17.14f}".rstrip('0')}'
        val2_str = f'{f"{0:^17d}" if val2 == 0 else f"{val2:17.14f}".rstrip('0')}'
        lines += f"{ind} {val1_str:17s} {equality} {val2_str:17s}\n"

    header = "=========== Summary ==========\n"
    header += "tens1 stats:\n"
    header += "------------\n"
    header += f" Dimensions: {tens1.shape}\n"
    header += f" Num of zeros in tens1: {zero_count}\n"
    header += "comparison summary\n"
    header += "------------------\n"
    header += f" Tolerance for compariing values: {tol}\n"
    header += f"{' Num of matches:':40s} {match_count:3d}\n"
    header += f"{' Num of differences:':40s} {diff_count:3d}\n"
    header += f"{' Num of differences where tens1 is 0:':40s} {zero_diff_count:3d}\n"
    header += "\n========== Index by Index comparison ==========\n"
    
    return header + lines            

# Ne full tensors

In [ ]:
std_tensor_vmat = tor.grab_tensor_from_std(stdout_ne, 'VF[mnij]')
def_tensor_vmat = tor.grab_tensor_from_def(defout_ne, 'vmat')
std_tensor_bmat = tor.grab_tensor_from_std(stdout_ne, 'BF[mnij]')
def_tensor_bmat = tor.grab_tensor_from_def(defout_ne, 'bmat')
std_tensor_xmat = tor.grab_tensor_from_std(stdout_ne, 'XF[mnij]')
def_tensor_xmat = tor.grab_tensor_from_def(defout_ne, 'xmat')
name = 'ne'

## VMAT

In [ ]:
def_full_vmat = tor.convert_to_full(def_tensor_vmat, std_tensor_vmat.shape)


matname = 'vmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_vmat, std_tensor_vmat)
    )


## BMAT

In [ ]:
def_full_bmat = tor.convert_to_full(def_tensor_bmat, std_tensor_bmat.shape)

matname = 'bmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_bmat, std_tensor_bmat)
    )


## XMAT

In [ ]:
def_full_xmat = tor.convert_to_full(def_tensor_xmat, std_tensor_xmat.shape)
#, skip_match=True)
matname = 'xmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_xmat, std_tensor_xmat)
    )


# NH3 full tensors

In [ ]:
std_tensor_vmat = tor.grab_tensor_from_std(stdout_nh3, 'VF[mnij]')
def_tensor_vmat = tor.grab_tensor_from_def(defout_nh3, 'vmat')
std_tensor_bmat = tor.grab_tensor_from_std(stdout_nh3, 'BF[mnij]')
def_tensor_bmat = tor.grab_tensor_from_def(defout_nh3, 'bmat')
std_tensor_xmat = tor.grab_tensor_from_std(stdout_nh3, 'XF[mnij]')
def_tensor_xmat = tor.grab_tensor_from_def(defout_nh3, 'xmat')
name = 'nh3'

## VMAT

In [ ]:
def_full_vmat = tor.convert_to_full(def_tensor_vmat, std_tensor_vmat.shape)


matname = 'vmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_vmat, std_tensor_vmat)
    )


## BMAT

In [ ]:
def_full_bmat = tor.convert_to_full(def_tensor_bmat, std_tensor_bmat.shape)

matname = 'bmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_bmat, std_tensor_bmat)
    )


## XMAT

In [ ]:
def_full_xmat = tor.convert_to_full(def_tensor_xmat, std_tensor_xmat.shape)
#, skip_match=True)
matname = 'xmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_xmat, std_tensor_xmat)
    )


# Cu full tensors

In [ ]:
std_tensor_vmat = tor.grab_tensor_from_std(stdout_cu, 'VF[mnij]')
def_tensor_vmat = tor.grab_tensor_from_def(defout_cu, 'vmat')
std_tensor_bmat = tor.grab_tensor_from_std(stdout_cu, 'BF[mnij]')
def_tensor_bmat = tor.grab_tensor_from_def(defout_cu, 'bmat')
std_tensor_xmat = tor.grab_tensor_from_std(stdout_cu, 'XF[mnij]')
def_tensor_xmat = tor.grab_tensor_from_def(defout_cu, 'xmat')
name = 'cu'

## VMAT

In [ ]:
def_full_vmat = tor.convert_to_full(def_tensor_vmat, std_tensor_vmat.shape)


matname = 'vmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_vmat, std_tensor_vmat)
    )


## BMAT

In [ ]:
def_full_bmat = tor.convert_to_full(def_tensor_bmat, std_tensor_bmat.shape)

matname = 'bmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_bmat, std_tensor_bmat)
    )


## XMAT

In [ ]:
def_full_xmat = tor.convert_to_full(def_tensor_xmat, std_tensor_xmat.shape)
#, skip_match=True)
matname = 'xmat'

fname = f'comp_between_default_and_standard_{name}/compare_{matname}.txt'
with open(fname, 'w') as out:
    out.write(f'SYSTEM: {name.upper()}; TENSOR: {matname.upper()}\n')
    out.write(
        compare_side_by_side(def_full_xmat, std_tensor_xmat)
    )


# Ne

In [ ]:
std_tensor = tor.grab_tensor_from_std(stdout_ne, 'VF[mnij]')
def_tensor = tor.grab_tensor_from_def(defout_ne, 'vmat')

## Std tensor print

In [ ]:
i = 0
for index, value in np.ndenumerate(std_tensor):
    if index[:2] == index[2:] or index[:2] == index[:1:-1]:
        print(f"{i:3d}", index, value)
        i += 1

## Default tensor print

In [ ]:


combined_tensor1 = 0.5 * (def_tensor[0,:,:] + def_tensor[1,:,:])
for index, value in np.ndenumerate(combined_tensor1):
    #print(np.array(index))
    if value:
        print(np.array(index), value)
print("====")
combined_tensor2 = 0.5 * (def_tensor[0,:,:] - def_tensor[1,:,:])
for index, value in np.ndenumerate(combined_tensor2):
    #print(np.array(index))
    if value:
        print(np.array(index), value)

## Compare

In [ ]:
compare(combined_tensor1, std_tensor, tol=1e-6)


In [ ]:
compare(combined_tensor2, std_tensor, tol=1e-6)

# NH3

In [ ]:
std_tensor_vmat = tor.grab_tensor_from_std(stdout_nh3, 'VF[mnij]')
def_tensor_vmat = tor.grab_tensor_from_def(defout_nh3, 'vmat')
std_tensor_bmat = tor.grab_tensor_from_std(stdout_nh3, 'BF[mnij]')
def_tensor_bmat = tor.grab_tensor_from_def(defout_nh3, 'bmat')
std_tensor_xmat = tor.grab_tensor_from_std(stdout_nh3, 'XF[mnij]')
def_tensor_xmat = tor.grab_tensor_from_def(defout_nh3, 'xmat')

## VMAT

### Std tensor print

In [ ]:
i = 0
for index, value in np.ndenumerate(std_tensor_vmat):
    if value > 1e-13:#index[:2] == index[2:] or index[:2] == index[:1:-1]:
        print(index, value)
        i += 1
print(i)

### Default tensor print

In [ ]:
print("==== V+")
combined_tensor1 = 0.5 * (def_tensor_vmat[0,:,:] + def_tensor_vmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor1):
    #print(np.array(index))
    if value:
        print(np.array(index), value)
print("==== V-")
combined_tensor2 = 0.5 * (def_tensor_vmat[0,:,:] - def_tensor_vmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor2):
    #print(np.array(index))
    if value:
        print(np.array(index), value)

### Compare

In [ ]:
compare(combined_tensor1, std_tensor_vmat, tol=1e-6)

In [ ]:
compare(combined_tensor2, std_tensor_vmat, tol=1e-5)

## BMAT

In [ ]:
i = 0
for index, value in np.ndenumerate(std_tensor_bmat):
    if index[:2] == index[2:] or index[:2] == index[:1:-1]:
        print(index, value)
        i += 1
print(i)

### Default tensor print

In [ ]:
print("==== B+")
combined_tensor1 = 0.5 * (def_tensor_bmat[0,:,:] + def_tensor_bmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor1):
    #print(np.array(index))
    if value:
        print(np.array(index), value)
print("==== B-")
combined_tensor2 = 0.5 * (def_tensor_bmat[0,:,:] - def_tensor_bmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor2):
    #print(np.array(index))
    if value:
        print(np.array(index), value)

### Compare

In [ ]:
compare(combined_tensor1, std_tensor_bmat, tol=1e-6)


In [ ]:
compare(combined_tensor2, std_tensor_bmat, tol=1e-6)

# Cu

In [ ]:
std_tensor_vmat = tor.grab_tensor_from_std(stdout_cu, 'VF[mnij]')
def_tensor_vmat = tor.grab_tensor_from_def(defout_cu, 'vmat')
std_tensor_bmat = tor.grab_tensor_from_std(stdout_cu, 'BF[mnij]')
def_tensor_bmat = tor.grab_tensor_from_def(defout_cu, 'bmat')
std_tensor_xmat = tor.grab_tensor_from_std(stdout_cu, 'XF[mnij]')
def_tensor_xmat = tor.grab_tensor_from_def(defout_cu, 'xmat')

## VMAT

### Std tensor print

In [ ]:
i = 0
for index, value in np.ndenumerate(std_tensor_vmat):
    if value > 1e-13:#index[:2] == index[2:] or index[:2] == index[:1:-1]:
        print(index, value)
        i += 1
print(i)

### Default tensor print

In [ ]:
combined_tensor1 = 0.5 * (def_tensor_vmat[0,:,:] + def_tensor_vmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor1):
    #print(np.array(index))
    if value:
        print(np.array(index), value)
print("====")
combined_tensor2 = 0.5 * (def_tensor_vmat[0,:,:] - def_tensor_vmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor2):
    #print(np.array(index))
    if value:
        print(np.array(index), value)

### Compare

In [ ]:
compare(combined_tensor1, std_tensor_vmat, tol=1e-6)


In [ ]:
compare(combined_tensor2, std_tensor_vmat, tol=1e-6)

## BMAT

In [ ]:
i = 0
for index, value in np.ndenumerate(std_tensor_bmat):
    if index[:2] == index[2:] or index[:2] == index[:1:-1]:
        print(index, value)
        i += 1
print(i)

### Default tensor print

In [ ]:
combined_tensor1 = 0.5 * (def_tensor_bmat[0,:,:] + def_tensor_bmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor1):
    #print(np.array(index))
    if value:
        print(np.array(index), value)
print("====")
combined_tensor2 = 0.5 * (def_tensor_bmat[0,:,:] - def_tensor_bmat[1,:,:])
for index, value in np.ndenumerate(combined_tensor2):
    #print(np.array(index))
    if value:
        print(np.array(index), value)

### Compare

In [ ]:
compare(combined_tensor1, std_tensor_bmat, tol=1e-6)


In [ ]:
compare(combined_tensor2, std_tensor_bmat, tol=1e-6)